In [1]:
import pandas as pd

In [2]:
p = pd.read_csv('python/benchmarks_py.csv')

In [3]:
p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Problem    26 non-null     int64  
 1   Date       26 non-null     object 
 2   Benchmark  26 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 756.0+ bytes


In [4]:
p.groupby(by = ['Problem', 'Date']).min()

,,Benchmark
Problem,Date,
1,2024-10-16,0.000013
2,2024-10-16,0.000006
3,2024-10-16,0.001406
4,2024-10-16,0.121776
5,2024-10-16,6.802064
6,2024-10-16,0.000004
7,2024-10-16,0.023532
8,2024-10-16,0.000923
9,2024-10-16,8.574732
